SMB、HML两个指标的计算方式如下：
1. 首先，按市值大小平均分为两组(Small 组, Big 组)，基准是这一时间的市场上公司市值中位数;
2. 按 BM 从小到大分三组,即前 30%(Growth 组),中间 40%(Neutral 组),后 30%(Value 组)；
3. 每个组的月回报以组内所有成员股票当月回报的加权平均数为依据,计算每个月的 SMB 和 HML 值。具体计算公式如下: 
   SMB = Small size return - Big size return)
   HML = Value company return - Growth company return
4. 以市场回报减去无风险回报,即得到超额市场回报(Rm-Rf)。（由于Ricequant平台无法提取国债收益率，我只能对这个时段的所有IBO1M求平均值作为无风    险回报率

# %pdb

# %load fama.py

In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

In [2]:
research_starttime='2010-04-01'
research_endtime='2018-04-01'
format='%Y-%m-%d %H:%M:%S'
data_type=""#"net"
data_mode=""#"fast"
data_mode_global=data_mode

### 定义基础数据获取函数

In [3]:
## 从网络获取数据
#market_cap 指定证券收盘价 x 总股本 x 交易币种兑人民币汇率
def get_dp():
        interval_month=24
        # date_range=pd.date_range(research_starttime,research_endtime)
        # len(date_range)/31
        # pd.to_datetime(research_endtime)-(pd.to_datetime(research_endtime)-pd.to_datetime(research_starttime))/4
        # print(datetime.date.today() +relativedelta(months=-1))
        step0=pd.to_datetime(research_endtime)
        step1=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*1)
        step2=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*2)
        step3=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*3)
        step4=pd.to_datetime(research_endtime)-relativedelta(months=interval_month*4)
        
        #网络请求API
        def get_years_fundamentals(interval_month,enddate):
            return get_fundamentals(
                query(fundamentals.eod_derivative_indicator.market_cap,fundamentals.financial_indicator.book_value_per_share,fundamentals.financial_indicator.return_on_equity,fundamentals.balance_sheet.total_assets)
                ,enddate, str(interval_month)+'m')
        
        # arr data
        dp_arr = [get_years_fundamentals(interval_month,step0),
                  get_years_fundamentals(interval_month,step1),
                  get_years_fundamentals(interval_month,step2),
                  get_years_fundamentals(interval_month,step3)]
        dp_arr
        
        # merge data 
        dp_merge=pd.concat(dp_arr,axis=1)#,ignore_index=True
        return dp_merge
        
#转换索引
def trans_index(df):
    if(type(df.index[0])==str):
        df.index=pd.to_datetime(df.index)
    return df
        
#get_all_instruments
def all_instruments_cs():
    df_market=None
    if(data_type=="net"):
        df_market = all_instruments(type='CS')
    else:
        df_market=pd.read_csv("all_instruments.csv",index_col=0)
    return df_market

#get_price_cs
def get_price_cs(data_starttime,data_endtime):
    df_price_data=None
    df_market=all_instruments_cs()
    if(data_type=="net"):
        df_price_data = get_price(list(df_market['order_book_id']),start_date=data_starttime, end_date=data_endtime,frequency='1d',fields='close')
    else:
        df_price_data=pd.read_csv("price_data.csv",index_col=0)
        df_price_data=trans_index(df_price_data)
    return df_price_data

#转换Series
def trans_series(df,key=1):
    if(type(df)==pd.DataFrame):
        df=df[key]
    return df


In [4]:
df_book_value=None
df_market_cap=None
df_return_on_equity=None
df_total_assets=None

#网络获取方式
if(data_type=="net"):
    dp_merge=get_dp()
    df_book_value=dp_merge["book_value_per_share"]
    df_market_cap=dp_merge["market_cap"]
    df_return_on_equity=dp_merge["return_on_equity"]
    df_total_assets=dp_merge["total_assets"]
    
    df_book_value=df_book_value[::-1]
    df_market_cap=df_market_cap[::-1]    
    df_return_on_equity=df_return_on_equity[::-1]    
    df_total_assets=df_total_assets[::-1]    
else:
#本地获取方式
    df_book_value=pd.read_csv("./book_value_per_share.csv",index_col=0)
    df_market_cap=pd.read_csv("./market_cap.csv",index_col=0)
    df_return_on_equity=pd.read_csv("./return_on_equity.csv",index_col=0)
    df_total_assets=pd.read_csv("./total_assets.csv",index_col=0)
    df_book_value=trans_index(df_book_value)
    df_market_cap=trans_index(df_market_cap)
    df_return_on_equity=trans_index(df_return_on_equity)    
    df_total_assets=trans_index(df_total_assets)    
    


In [5]:
df_book_value

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,6.59,3.4000,0.7033,0.8100,4.1600,-0.1580,0.9956,1.8400,0.3100,1.1098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,7.12,3.5500,0.7274,0.7600,4.2200,-0.3681,0.9947,2.0800,0.3100,1.1148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,7.12,3.5500,0.7274,0.7600,4.2200,-0.3681,0.9947,2.0800,0.3100,1.1148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,7.12,3.5500,0.7274,0.7600,4.2200,-0.3681,0.9947,2.0800,0.3100,1.1148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,7.12,3.5500,0.7274,0.7600,4.2200,-0.3681,0.9947,2.0800,0.3100,1.3221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,8.73,3.6200,0.7707,0.7510,2.6500,-0.3113,0.9958,2.0400,0.2974,1.3221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,8.73,3.6200,0.7707,0.7510,2.6500,-0.3113,0.9958,2.0400,0.2974,1.3221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,9.22,3.6600,0.8411,0.7390,2.7500,-0.3448,0.9967,2.2200,0.3130,1.3377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,9.22,3.6600,0.8411,0.7390,2.7500,-0.3448,0.9967,2.2200,0.3130,1.3377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,9.22,3.6600,0.8411,0.7390,2.7500,-0.3448,0.9967,2.2200,0.3130,1.3377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_market_cap

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,7.276031e+10,1.074232e+11,9.531354e+08,5.550005e+09,6.207923e+09,1.433482e+09,1.014941e+09,1.337260e+10,2.109700e+09,7.342462e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,6.210868e+10,8.191432e+10,8.288499e+08,4.205935e+09,4.473356e+09,1.520415e+09,9.096171e+08,1.228185e+10,2.150865e+09,4.583079e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,5.372400e+10,8.004513e+10,7.725855e+08,3.858488e+09,4.153831e+09,1.333600e+09,1.013468e+09,1.055847e+10,2.150865e+09,4.189733e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,6.102259e+10,7.762618e+10,7.037246e+08,3.538471e+09,4.161439e+09,1.300307e+09,7.365321e+08,8.671467e+09,2.150865e+09,4.827431e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,6.102259e+10,8.796168e+10,8.221317e+08,4.169361e+09,6.808935e+09,1.385391e+09,8.389100e+08,1.104930e+10,2.150865e+09,5.608163e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,6.151049e+10,9.422895e+10,9.497763e+08,4.370515e+09,6.162277e+09,1.451978e+09,9.479168e+08,1.360166e+10,2.150865e+09,5.208857e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,6.001194e+10,9.279957e+10,9.606933e+08,4.004781e+09,5.690596e+09,1.524115e+09,9.928452e+08,1.323080e+10,2.150865e+09,4.946626e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,6.318330e+10,1.006062e+11,1.137884e+09,4.096215e+09,5.629734e+09,1.646192e+09,1.024516e+09,1.902269e+10,2.150865e+09,5.125420e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,5.694512e+10,8.906120e+10,1.039631e+09,3.492754e+09,5.789497e+09,1.562957e+09,8.912038e+08,1.773560e+10,2.150865e+09,4.261251e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,5.642237e+10,9.928675e+10,1.008560e+09,3.465324e+09,6.101415e+09,1.622146e+09,8.750001e+08,1.657941e+10,2.150865e+09,4.714195e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_return_on_equity

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,27.2887,15.3888,-3.5235,-1.5711,17.5935,NaN,0.4284,13.5694,16.4939,15.7353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,7.4126,2.9509,-0.1331,-0.9862,1.7952,NaN,-0.0824,3.8134,-2.9945,0.4284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,7.4126,2.9509,-0.1331,-0.9862,1.7952,NaN,-0.0824,3.8134,-2.9945,0.4284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,7.4126,2.9509,-0.1331,-0.9862,1.7952,NaN,-0.0824,3.8134,-2.9945,0.4284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,7.4126,2.9509,-0.1331,-0.9862,1.7952,NaN,-0.0824,3.8134,-2.9945,17.4904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,11.9201,7.2911,5.5388,-2.3054,4.8008,NaN,0.0230,8.9457,-7.7647,17.4904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,11.9201,7.2911,5.5388,-2.3054,4.8008,NaN,0.0230,8.9457,-7.7647,17.4904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,17.9951,8.4296,14.0997,-3.8508,6.2662,NaN,0.1195,11.4174,-2.6648,18.5447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,17.9951,8.4296,14.0997,-3.8508,6.2662,NaN,0.1195,11.4174,-2.6648,18.5447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,17.9951,8.4296,14.0997,-3.8508,6.2662,NaN,0.1195,11.4174,-2.6648,18.5447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_total_assets

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,5.878110e+11,1.376086e+11,1.842607e+08,1.329657e+09,7.272028e+09,4.219999e+08,7.940798e+07,7.502010e+09,2.398579e+08,2.834418e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,6.199276e+11,1.483960e+11,2.182259e+08,1.308876e+09,7.701140e+09,3.458523e+08,7.983640e+07,9.259596e+09,2.150322e+08,2.907826e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,6.199276e+11,1.483960e+11,2.182259e+08,1.308876e+09,7.701140e+09,3.458523e+08,7.983640e+07,9.259596e+09,2.150322e+08,2.907826e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,6.199276e+11,1.483960e+11,2.182259e+08,1.308876e+09,7.701140e+09,3.458523e+08,7.983640e+07,9.259596e+09,2.150322e+08,2.907826e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,6.199276e+11,1.483960e+11,2.182259e+08,1.308876e+09,7.701140e+09,3.458523e+08,7.983640e+07,9.259596e+09,2.150322e+08,2.769294e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,6.243982e+11,1.605127e+11,2.256770e+08,1.316129e+09,7.533026e+09,3.488154e+08,7.735679e+07,9.029187e+09,2.158706e+08,2.769294e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,6.243982e+11,1.605127e+11,2.256770e+08,1.316129e+09,7.533026e+09,3.488154e+08,7.735679e+07,9.029187e+09,2.158706e+08,2.769294e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,6.750639e+11,1.879353e+11,1.860109e+08,1.299482e+09,8.054597e+09,2.943772e+08,7.754770e+07,9.767944e+09,2.210542e+08,2.895754e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,6.750639e+11,1.879353e+11,1.860109e+08,1.299482e+09,8.054597e+09,2.943772e+08,7.754770e+07,9.767944e+09,2.210542e+08,2.895754e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,6.750639e+11,1.879353e+11,1.860109e+08,1.299482e+09,8.054597e+09,2.943772e+08,7.754770e+07,9.767944e+09,2.210542e+08,2.895754e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


计算说明：
dp_merge.major_xs("2018-03-30")


In [9]:
# df_book_value=df_book_value[::-1]
# df_book_value0=df_book_value.dropna(axis=1,how="all")
# df_book_value=df_book_value0.sort_index()
index_date = df_book_value.index
stocks = df_book_value.columns
data_count=len(index_date)
data_starttime=index_date[0]
data_endtime=index_date[-1]
data_starttime,data_endtime

(Timestamp('2010-04-02 00:00:00'), Timestamp('2018-03-30 00:00:00'))

In [10]:
#all_instruments_cs
df_market = all_instruments_cs()
print("df_market['order_book_id']=",len(df_market['order_book_id']),df_market['order_book_id']);

df_market['order_book_id']= 3631 0       002322.XSHE
1       601877.XSHG
2       002480.XSHE
3       002735.XSHE
4       000839.XSHE
5       603966.XSHG
6       300521.XSHE
7       002202.XSHE
8       300160.XSHE
9       002025.XSHE
10      002011.XSHE
11      600261.XSHG
12      000759.XSHE
13      603421.XSHG
14      600635.XSHG
15      300151.XSHE
16      000776.XSHE
17      002702.XSHE
18      000968.XSHE
19      002013.XSHE
20      000756.XSHE
21      600672.XSHG
22      300095.XSHE
23      300553.XSHE
24      002718.XSHE
25      603089.XSHG
26      002205.XSHE
27      000951.XSHE
28      300302.XSHE
29      300515.XSHE
           ...     
3601    002490.XSHE
3602    603685.XSHG
3603    002156.XSHE
3604    600966.XSHG
3605    600606.XSHG
3606    002453.XSHE
3607    300255.XSHE
3608    603458.XSHG
3609    300379.XSHE
3610    000875.XSHE
3611    000667.XSHE
3612    600583.XSHG
3613    300223.XSHE
3614    002569.XSHE
3615    300359.XSHE
3616    300050.XSHE
3617    002157.XSHE
3618   

In [11]:
df_price_data0 = get_price_cs(data_starttime,data_endtime)#columns是乱序
df_price_data0#3631
# df_price_data#3514
# df_price_data0['000001.XSHE']

,603186.XSHG,000062.XSHE,300643.XSHE,600315.XSHG,300146.XSHE,000990.XSHE,300631.XSHE,300363.XSHE,600551.XSHG,603656.XSHG,...,601857.XSHG,000825.XSHE,000543.XSHE,002917.XSHE,000569.XSHE,000732.XSHE,600696.XSHG,603288.XSHG,002725.XSHE,002246.XSHE
2010-04-02,NaN,9.6163,NaN,18.0081,NaN,14.8820,NaN,NaN,15.5627,NaN,...,10.5334,7.7195,4.9641,NaN,NaN,1.9866,7.22,NaN,NaN,11.6093
2010-04-06,NaN,9.5244,NaN,18.5700,NaN,15.1620,NaN,NaN,15.2372,NaN,...,10.5334,7.6462,5.0109,NaN,NaN,1.9866,7.17,NaN,NaN,11.7370
2010-04-07,NaN,9.4234,NaN,18.5747,NaN,15.5483,NaN,NaN,15.3205,NaN,...,10.4766,7.8111,4.9589,NaN,NaN,1.9866,7.22,NaN,NaN,11.9540
2010-04-08,NaN,9.4693,NaN,19.1133,NaN,15.6449,NaN,NaN,15.8049,NaN,...,10.3548,7.5913,4.8860,NaN,NaN,1.9866,7.13,NaN,NaN,11.7498
2010-04-09,NaN,9.6347,NaN,19.2201,NaN,15.7801,NaN,NaN,15.8503,NaN,...,10.3954,7.6462,4.9173,NaN,NaN,1.9866,7.15,NaN,NaN,12.1774
2010-04-12,NaN,9.7724,NaN,19.9120,NaN,16.1567,NaN,NaN,16.4786,NaN,...,10.3548,7.4173,4.8444,NaN,NaN,1.9866,7.00,NaN,NaN,12.1774
2010-04-13,NaN,9.4785,NaN,20.1860,NaN,15.6932,NaN,NaN,16.4862,NaN,...,10.5902,7.4997,4.7612,NaN,NaN,1.9866,6.87,NaN,NaN,11.9795
2010-04-14,NaN,10.4245,NaN,19.8335,NaN,15.7608,NaN,NaN,17.1674,NaN,...,10.5253,7.5089,4.7872,NaN,NaN,1.9866,6.94,NaN,NaN,11.8455
2010-04-15,NaN,10.1857,NaN,19.4014,NaN,15.9153,NaN,NaN,16.7662,NaN,...,10.7363,7.4081,4.7351,NaN,NaN,1.9866,6.77,NaN,NaN,11.6349
2010-04-16,NaN,9.9561,NaN,20.1074,NaN,16.6106,NaN,NaN,16.4634,NaN,...,10.5821,7.3440,4.7039,NaN,NaN,1.9866,6.71,NaN,NaN,11.7306


In [12]:
df_price_data=df_price_data0.reindex(index_date,columns=stocks)#月度价格数据
df_price_data

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,7.9167,7.8846,11.35,6.07,4.0482,5.1667,1.5816,6.4584,3.5875,11.5412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,6.7577,6.0123,9.87,4.60,2.9171,5.4800,1.4175,5.9316,3.6575,7.2039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,5.8455,5.9351,9.20,4.22,2.7458,4.8067,1.5794,5.0993,3.6575,6.5856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,5.9164,5.7558,8.38,3.87,2.7508,4.6867,1.1478,4.1880,3.6575,7.5879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,5.9164,6.5221,9.79,4.56,4.5008,4.9933,1.3073,5.3364,3.6575,8.8151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,5.9637,6.9868,11.31,4.78,4.0734,5.2333,1.4772,6.5908,3.6575,8.1875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,5.8184,6.8809,11.44,4.38,3.7616,5.4933,1.5472,6.4111,3.6575,7.7753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,6.1259,7.4597,13.55,4.48,3.7214,5.9333,1.5966,9.2176,3.6575,8.0563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,5.5211,6.6037,12.38,3.82,3.8270,5.6333,1.3888,8.5939,3.6575,6.6980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,5.4704,7.3619,12.01,3.79,4.0331,5.8467,1.3636,8.0337,3.6575,7.4100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#save data to reuse offline
# df_book_value.to_csv("book_value_per_share.csv")
# df_market_cap.to_csv("market_cap.csv")
# df_market.to_csv("all_instruments.csv")
# df_price_data.to_csv("price_data.csv")


In [14]:
#btm 函数定义
def get_btm(df_book_value,df_price_data):
    btm=df_book_value/df_price_data#3631
#     btm2=btm.dropna(axis=1,how="all")
    # b2m2[~np.isnan(b2m2['000003.XSHE'])]
    return btm     

In [15]:
#需要传入月度数据df_price_data
btm=get_btm(df_book_value,df_price_data)
btm

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,0.832418,0.431220,0.061965,0.133443,1.027617,-0.030580,0.629489,0.284900,0.086411,0.096160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,1.053613,0.590456,0.073698,0.165217,1.446642,-0.067172,0.701728,0.350664,0.084757,0.154750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,1.218031,0.598137,0.079065,0.180095,1.536893,-0.076581,0.629796,0.407899,0.084757,0.169278,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,1.203435,0.616769,0.086802,0.196382,1.534099,-0.078541,0.866614,0.496657,0.084757,0.146918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,1.203435,0.544303,0.074300,0.166667,0.937611,-0.073719,0.760881,0.389776,0.084757,0.149981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,1.463856,0.518120,0.068143,0.157113,0.650562,-0.059484,0.674113,0.309522,0.081312,0.161478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,1.500412,0.526094,0.067369,0.171461,0.704487,-0.056669,0.643614,0.318198,0.081312,0.170038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,1.505085,0.490636,0.062074,0.164955,0.738969,-0.058113,0.624264,0.240844,0.085578,0.166044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,1.669957,0.554235,0.067940,0.193455,0.718579,-0.061207,0.717670,0.258323,0.085578,0.199716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,1.685434,0.497154,0.070033,0.194987,0.681858,-0.058973,0.730933,0.276336,0.085578,0.180526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#是否需要做价格回溯，即不存在时，往前找
def get_return_data(index_date,df_price_data):
    df_price_data2=df_price_data.reindex(index_date)
    df_price_data3=df_price_data2.sort_index()
    df_price_data4=df_price_data3.pct_change()
#     df_price_data5=df_price_data4.dropna(axis=1,how="all")
#     df_price_data6=df_price_data5.sort_index(ascending=False)
    return df_price_data4
# df_price_data6


In [17]:
# return_data=None
# if(data_mode=="fast"):
#     return_data=pd.read_csv("return_data.csv",index_col=0)
#     return_data=trans_index(return_data)
# else:
#     return_data=get_return_data(index_date,stocks,df_price_data)
return_data=get_return_data(index_date,df_price_data)
return_data

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603991.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
2010-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-04,-0.146399,-0.237463,-0.130396,-0.242175,-0.279408,0.060638,-0.103756,-0.081568,0.019512,-0.375810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-04,-0.134987,-0.012840,-0.067882,-0.082609,-0.058723,-0.122865,0.114215,-0.140316,0.000000,-0.085829,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-06,0.012129,-0.030210,-0.089130,-0.082938,0.001821,-0.024965,-0.273268,-0.178711,0.000000,0.152196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-06,0.000000,0.133135,0.168258,0.178295,0.636179,0.065419,0.138961,0.274212,0.000000,0.161731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-08,0.007995,0.071250,0.155260,0.048246,-0.094961,0.048064,0.129963,0.235065,0.000000,-0.071196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,-0.024364,-0.015157,0.011494,-0.083682,-0.076545,0.049682,0.047387,-0.027265,0.000000,-0.050345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-10,0.052850,0.084117,0.184441,0.022831,-0.010687,0.080098,0.031929,0.437756,0.000000,0.036140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,-0.098728,-0.114750,-0.086347,-0.147321,0.028376,-0.050562,-0.130152,-0.067664,0.000000,-0.168601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-11,-0.009183,0.114814,-0.029887,-0.007853,0.053854,0.037882,-0.018145,-0.065186,0.000000,0.106300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# return_data.to_csv("return_data.csv")

In [66]:
def cumulative_cap(date,stk_market_cap,valid_total_market_cap):
    if(date in valid_total_market_cap):
        valid_total_market_cap[date]=valid_total_market_cap[date]+stk_market_cap
    else:
        valid_total_market_cap[date]=0.0

def recordnan_withstk(stk,date,nan_map):
    datestr=date.strftime("%F")
    nan_skt_map=nan_map.get(stk)
    if(nan_skt_map):
        if(datestr not in nan_skt_map):
            nan_skt_map.append(datestr)
    else:
        nan_map[stk]=[datestr]
        
def recordnan_withdate(stk,date,nan_map):
    datestr=date.strftime("%F")
    nan_skt_map=nan_map.get(datestr)
    if(nan_skt_map):
        if(stk not in nan_skt_map):
            nan_skt_map.append(stk)
    else:
        nan_map[datestr]=[stk]
#获得二分组
def get_2group(index_date,factor_data,recordnan):  
    median_size = {}
    smallgroup = {}
    middlegroup = {}
    biggroup = {}
    nan_map={}
    stocks=factor_data.columns
    for date in index_date:
        median_size[date] = np.median(factor_data.loc[date].dropna())
        factor_data_dropna=factor_data.loc[date].dropna()
        if(len(factor_data_dropna)>0):
            median_size[date] = np.median(factor_data_dropna)
        else:
            median_size[date]=0
    for date in index_date:
        smallgroup[date]=[]
        middlegroup[date]=[]
        biggroup[date]=[]
        for stk in stocks:
            if(np.isnan(factor_data[stk][date])):
                recordnan(stk,date,nan_map)
                continue
            #计算factor分组
            if factor_data[stk][date]<median_size[date]:
                smallgroup[date].append(stk)
            elif factor_data[stk][date]>median_size[date]:
                biggroup[date].append(stk)
            elif factor_data[stk][date]==median_size[date]:
                middlegroup[date].append(stk)
            else:
                recordnan(stk,date,nan_map)

    return {"smallgroup":smallgroup,"middlegroup":middlegroup,"biggroup":biggroup,
            "median_size":median_size,"nan_map":nan_map}
        
#获得三分组
def get_3group(index_date,factor_data,recordnan):
    lmark = {}
    hmark = {}
    smallgroup = {}
    middlegroup = {}
    biggroup = {}
    nan_map={}
    stocks=factor_data.columns
    for date in index_date:
        factor_data_dropna=factor_data.loc[date].dropna()
        if(len(factor_data_dropna)>0):
            lmark[date] = np.percentile(factor_data_dropna,30)
            hmark[date] = np.percentile(factor_data_dropna,70)
        else:
            lmark[date]=0
            hmark[date]=0
    for date in index_date:
        smallgroup[date]=[]
        middlegroup[date]=[]
        biggroup[date]=[]
        for stk in stocks:
            if(np.isnan(factor_data[stk][date])):
                recordnan(stk,date,nan_map)
                continue
            #计算factor分组
            if factor_data[stk][date]<=lmark[date]:
                smallgroup[date].append(stk)
                
            elif factor_data[stk][date]>=hmark[date]:
                biggroup[date].append(stk)

            elif(lmark[date]<factor_data[stk][date]<hmark[date]):#middle
                middlegroup[date].append(stk)
            else:
                recordnan(stk,date,nan_map)

    return {"smallgroup":smallgroup,"middlegroup":middlegroup,"biggroup":biggroup,
            "lmark":lmark,"hmark":hmark,"nan_map":nan_map}

#计算每一小组总回报
def get_group_return(index_date,group,return_data,df_market_cap,nan_map,valid_total_market_cap,recordnan=recordnan_withstk):
    group_return={}
    for date in index_date:
        group_return[date]=0.0
        group_stocks=group[date]
        
        for stk in group_stocks:
            if(np.isnan(df_market_cap[stk][date]) or np.isnan(return_data[stk][date])):
                recordnan(stk,date,nan_map)
                continue
            group_return[date]=group_return[date]+return_data[stk][date]*df_market_cap[stk][date]
            cumulative_cap(date,df_market_cap[stk][date],valid_total_market_cap)
    return  group_return
    
#计算三分组因子#factor_type "smb/bms" 两分组的也能用
def get_3group_factor(index_date,factor_3group,return_data,df_market_cap,recordnan=recordnan_withstk,factor_type="smb"):
    factor = pd.Series(index = index_date)
    valid_total_market_cap={}
    nan_map={}
    #计算factor
    smallgroup_return=get_group_return(index_date,factor_3group["smallgroup"],return_data,df_market_cap,nan_map,valid_total_market_cap,recordnan)
#     middlegroup_return=get_group_return(index_date,factor_3group["middlegroup"],return_data,df_market_cap,nan_map,valid_total_market_cap,recordnan)
    biggroup_return=get_group_return(index_date,factor_3group["biggroup"],return_data,df_market_cap,nan_map,valid_total_market_cap,recordnan)
    
    for date in index_date:
        if(date not in valid_total_market_cap):
            factor[date]=0
            continue
            
        smallgroup=smallgroup_return[date]
        biggroup=biggroup_return[date]
        mktcap = valid_total_market_cap[date]
        netgroup=smallgroup - biggroup
        if(factor_type=="bms"):
            netgroup=biggroup-smallgroup
        factor[date] = netgroup/mktcap#每日回报率＝每日总回报/总市值
        
    return factor

#获取hml因子
def get_factor_hml(index_date,factor_data,return_data,df_market_cap,recordnan=recordnan_withstk):
    if(data_mode=="fast"):
        factor=pd.read_csv("hml2.csv",index_col=0,header=None)
        factor=trans_series(factor)
        factor=trans_index(factor)
    else:
        factor_3group=get_3group(index_date,factor_data,recordnan)
        factor=get_3group_factor(index_date,factor_3group,return_data,df_market_cap,recordnan,factor_type="bms")
    return factor

#获取盈利因子rmw
def get_factor_rmw(index_date,factor_data,return_data,df_market_cap,recordnan=recordnan_withstk):
    if(data_mode=="fast"):
        factor=pd.read_csv("rmw.csv",index_col=0,header=None)
        factor=trans_series(factor)
        factor=trans_index(factor)
    else:
        factor_3group=get_3group(index_date,factor_data,recordnan)
        factor=get_3group_factor(index_date,factor_3group,return_data,df_market_cap,recordnan,factor_type="bms")
    return factor  
#获取投资风格因子cma
def get_factor_cma(index_date,factor_data,return_data,df_market_cap,recordnan=recordnan_withstk):
    if(data_mode=="fast"):
        factor=pd.read_csv("cma.csv",index_col=0,header=None)
        factor=trans_series(factor)
        factor=trans_index(factor)
    else:
        factor_3group=get_3group(index_date,factor_data,recordnan)
        factor=get_3group_factor(index_date,factor_3group,return_data,df_market_cap,recordnan,factor_type="smb")
    return factor 

#获取市值因子smb
def get_factor_smb(index_date,factor_data,return_data,df_market_cap,recordnan=recordnan_withstk):
    if(data_mode=="fast"):
        factor=pd.read_csv("smb.csv",index_col=0,header=None)
        factor=trans_series(factor)
        factor=trans_index(factor)
    else:
        factor_2group=get_2group(index_date,factor_data,recordnan)
        factor=get_3group_factor(index_date,factor_2group,return_data,df_market_cap,recordnan,factor_type="smb")
    return factor 

In [67]:
# hml=get_factor_hml(index_date,btm,return_data,df_market_cap,recordnan_withstk)
smb=get_factor_smb(index_date,df_market_cap,return_data,df_market_cap,recordnan_withstk)
smb

2010-04-02    0.000000
2010-05-04    0.072521
2010-06-04    0.063869
2010-07-06    0.033867
2010-08-06   -0.090692
2010-09-08   -0.017744
2010-10-08   -0.024794
2010-11-10   -0.130030
2010-12-10    0.060862
2011-01-11    0.005458
2011-02-11   -0.007782
2011-03-11   -0.030101
2011-04-13   -0.033197
2011-05-13    0.041192
2011-06-15    0.032963
2011-07-15   -0.051744
2011-08-16    0.049584
2011-09-16    0.036984
2011-10-18    0.024557
2011-11-18   -0.013176
2011-12-20    0.050213
2012-01-20   -0.049039
2012-02-29   -0.039565
2012-03-30    0.047447
2012-04-11   -0.014933
2012-05-11   -0.033474
2012-06-13    0.009621
2012-07-13    0.025162
2012-08-14    0.011435
2012-09-14    0.007234
                ...   
2015-10-30   -0.089938
2015-11-30   -0.025595
2015-12-31   -0.025909
2016-02-01    0.151607
2016-03-01   -0.014541
2016-04-01   -0.075211
2016-04-06   -0.010317
2016-05-06    0.028574
2016-06-08   -0.018613
2016-07-12   -0.044721
2016-08-12   -0.011503
2016-09-14    0.011056
2016-10-14 

In [22]:
rmw=get_factor_rmw(index_date,df_return_on_equity,return_data,df_market_cap,recordnan_withstk)
rmw

2010-04-02    0.000000
2010-05-04   -0.059653
2010-06-04   -0.055799
2010-07-06   -0.021232
2010-08-06    0.068714
2010-09-08    0.009074
2010-10-08    0.022023
2010-11-10    0.112015
2010-12-10   -0.047111
2011-01-11   -0.003005
2011-02-11    0.001837
2011-03-11    0.015860
2011-04-13    0.023954
2011-05-13   -0.032229
2011-06-15   -0.024353
2011-07-15    0.031788
2011-08-16   -0.022648
2011-09-16   -0.031021
2011-10-18   -0.014331
2011-11-18    0.006063
2011-12-20   -0.030521
2012-01-20    0.046103
2012-02-29    0.021519
2012-03-30   -0.034879
2012-04-11    0.006767
2012-05-11    0.028260
2012-06-13   -0.013471
2012-07-13   -0.011125
2012-08-14   -0.001020
2012-09-14   -0.007077
                ...   
2015-10-30    0.052309
2015-11-30    0.026915
2015-12-31    0.018101
2016-02-01   -0.110696
2016-03-01   -0.001176
2016-04-01    0.060940
2016-04-06    0.005070
2016-05-06   -0.005344
2016-06-08    0.021405
2016-07-12    0.027043
2016-08-12    0.023506
2016-09-14   -0.006605
2016-10-14 

In [54]:
#获得三分组year
def get_3group_year(index_date,factor_data,recordnan=recordnan_withstk):
    lmark = {}
    hmark = {}
    smallgroup = {}
    middlegroup = {}
    biggroup = {}
    nan_map={}
    stocks=factor_data.columns
    for date in index_date:
        factor_data_dropna=factor_data.loc[date].dropna()
        if(len(factor_data_dropna)>0):
            lmark[date] = np.percentile(factor_data_dropna,30)
            hmark[date] = np.percentile(factor_data_dropna,70)
        else:
            lmark[date]=0
            hmark[date]=0
    for date in index_date:
        smallgroup[date]=[]
        middlegroup[date]=[]
        biggroup[date]=[]
        for stk in stocks:
            if(np.isnan(factor_data[stk][date])):
                recordnan(stk,date,nan_map)
                continue
            #计算factor分组
            if factor_data[stk][date]<=lmark[date]:
                smallgroup[date].append(stk)
                
            elif factor_data[stk][date]>=hmark[date]:
                biggroup[date].append(stk)

            elif(lmark[date]<factor_data[stk][date]<hmark[date]):#middle
                middlegroup[date].append(stk)
            else:
                recordnan(stk,date,nan_map)

    return {"smallgroup":smallgroup,"middlegroup":middlegroup,"biggroup":biggroup,
            "lmark":lmark,"hmark":hmark,"nan_map":nan_map}

In [74]:
df_asset_growth = df_total_assets.pct_change(periods=12)
factor_data=df_asset_growth
cma=get_factor_cma(index_date,df_asset_growth,return_data,df_market_cap,recordnan_withstk)
cma

# dd=get_3group_year(index_date,factor_data,recordnan_withstk)

2010-04-02    0.000000
2010-05-04    0.000000
2010-06-04    0.000000
2010-07-06    0.000000
2010-08-06    0.000000
2010-09-08    0.000000
2010-10-08    0.000000
2010-11-10    0.000000
2010-12-10    0.000000
2011-01-11    0.000000
2011-02-11    0.000000
2011-03-11    0.000000
2011-04-13   -0.007224
2011-05-13    0.020267
2011-06-15    0.013084
2011-07-15   -0.058743
2011-08-16    0.005121
2011-09-16    0.034821
2011-10-18    0.027782
2011-11-18    0.000357
2011-12-20    0.013863
2012-01-20   -0.013198
2012-02-29   -0.027667
2012-03-30    0.020256
2012-04-11   -0.009407
2012-05-11   -0.024452
2012-06-13   -0.002426
2012-07-13   -0.003377
2012-08-14    0.008065
2012-09-14    0.005348
                ...   
2015-10-30   -0.047610
2015-11-30   -0.030947
2015-12-31   -0.013603
2016-02-01    0.051122
2016-03-01    0.005240
2016-04-01   -0.035842
2016-04-06   -0.007805
2016-05-06    0.008564
2016-06-08   -0.011418
2016-07-12   -0.003757
2016-08-12   -0.002698
2016-09-14   -0.000429
2016-10-14 

In [92]:
# date="2010-06-04"
# df_asset_growth.loc[date].dropna() #Series([], Name: 2010-06-04 00:00:00, dtype: float64)
# np.percentile([],30)
obj={'a':'11',"b":2}
list(zip(obj.values(),obj.keys()))

[('11', 'a'), (2, 'b')]

In [69]:
# dd.keys()
#dict_keys(['smallgroup', 'middlegroup', 'biggroup', 'lmark', 'hmark', 'nan_map'])


{Timestamp('2010-04-02 00:00:00'): 0,
 Timestamp('2010-05-04 00:00:00'): 0,
 Timestamp('2010-06-04 00:00:00'): 0,
 Timestamp('2010-07-06 00:00:00'): 0,
 Timestamp('2010-08-06 00:00:00'): 0,
 Timestamp('2010-09-08 00:00:00'): 0,
 Timestamp('2010-10-08 00:00:00'): 0,
 Timestamp('2010-11-10 00:00:00'): 0,
 Timestamp('2010-12-10 00:00:00'): 0,
 Timestamp('2011-01-11 00:00:00'): 0,
 Timestamp('2011-02-11 00:00:00'): 0,
 Timestamp('2011-03-11 00:00:00'): 0,
 Timestamp('2011-04-13 00:00:00'): 0.28519047840547174,
 Timestamp('2011-05-13 00:00:00'): 0.24790218231124328,
 Timestamp('2011-06-15 00:00:00'): 0.2557795880087078,
 Timestamp('2011-07-15 00:00:00'): 0.26267776358071354,
 Timestamp('2011-08-16 00:00:00'): 0.2704827290981094,
 Timestamp('2011-09-16 00:00:00'): 0.2468759439720551,
 Timestamp('2011-10-18 00:00:00'): 0.2588934131105718,
 Timestamp('2011-11-18 00:00:00'): 0.23965943532484027,
 Timestamp('2011-12-20 00:00:00'): 0.2458271903483035,
 Timestamp('2012-01-20 00:00:00'): 0.25385904

In [ ]:
#old 计算方式smb，
def get_smb(index_date,stocks,return_data,df_market_cap,recordnan):
    smb = pd.Series(index = index_date)
    if(data_mode=="fast"):
        smb=pd.read_csv("smb.csv",index_col=0,header=None)
        smb=trans_series(smb)
        smb=trans_index(smb)
    else:
        median_size = {}
        nan_map={}
        valid_total_market_cap={}
        for date in index_date:
            #日市值中位数
            median_size[date] = np.median(df_market_cap.loc[date].dropna())
        
        #计算smb
        for date in index_date:
            small_size=0.0
            big_size=0.0
            for stk in stocks:
#                 if(stk=="000001.XSHE"):
#                     print('--test--',df_market_cap[stk][date],return_data[stk][date],median_size[date])
                if(np.isnan(df_market_cap[stk][date]) or np.isnan(return_data[stk][date])):
                    recordnan(stk,date,nan_map)
                    continue
                #计算市值size分组
                if df_market_cap[stk][date]<median_size[date]:#判定为小市值
                    small_size = small_size + return_data[stk][date]*df_market_cap[stk][date]#计算每日总的回报＝回报率＊市值
                    # print("--small_size--",small_size,return_data[stk][date],df_market_cap[stk][date])
                    cumulative_cap(date,df_market_cap[stk][date],valid_total_market_cap)
                elif(df_market_cap[stk][date]>=median_size[date]): 
                    big_size = big_size + return_data[stk][date]*df_market_cap[stk][date]
                    cumulative_cap(date,df_market_cap[stk][date],valid_total_market_cap)
                else:
                    recordnan(stk,date,nan_map)
#             mktcap = np.sum(df_market_cap.loc[date])
#             smb[date] = (small_size - big_size)/mktcap#每日回报率＝每日总回报/总市值
            
            if(date not in valid_total_market_cap):
                smb[date]=0
                continue
            mktcap = valid_total_market_cap[date]
            smb[date] = (small_size - big_size)/mktcap#每日回报率＝每日总回报/总市值
#             print("smb[date]=====",small_size,big_size,mktcap,smb[date])   
#         print("nan_map[stk]=[date] isnan",nan_map)   
    return smb

#old 计算方式hml，
def get_hml(index_date,stocks,return_data,df_market_cap,btm,recordnan):
    hml = pd.Series(index = index_date)
    if(data_mode=="fast"):
        hml=pd.read_csv("hml2.csv",index_col=0,header=None)
        hml=trans_series(hml)
        hml=trans_index(hml)
    else:
        lmark = {}
        hmark = {}
        nan_map={}
        valid_total_market_cap={}
        for date in index_date:
            lmark[date] = np.percentile(btm.loc[date].dropna(),30)
            hmark[date] = np.percentile(btm.loc[date].dropna(),70)
        
        #计算hml
        for date in index_date:
            value_btm=0.0
            middle_btm=0.0
            growth_btm=0.0
            for stk in stocks:
                if(np.isnan(df_market_cap[stk][date]) or np.isnan(return_data[stk][date])):
                    recordnan(stk,date,nan_map)
                    continue
                #计算btm分组
                if btm[stk][date]<=lmark[date]:
                    growth_btm = growth_btm + return_data[stk][date]*df_market_cap[stk][date]
                    cumulative_cap(date,df_market_cap[stk][date],valid_total_market_cap)
                    
                elif btm[stk][date]>=hmark[date]:
                    value_btm = value_btm + return_data[stk][date]*df_market_cap[stk][date]
                    cumulative_cap(date,df_market_cap[stk][date],valid_total_market_cap)
                    
                elif(lmark[date]<btm[stk][date]<hmark[date]):#middle
                    middle_btm = middle_btm + return_data[stk][date]*df_market_cap[stk][date]
                else:
                    recordnan(stk,date,nan_map)
#1             df_stocks=df_market_cap[stocks]    
#2             mktcap = np.sum(df_market_cap.loc[date])
            if(date not in valid_total_market_cap):
                hml[date]=0
                continue
            mktcap = valid_total_market_cap[date]
            hml[date] = (value_btm - growth_btm)/mktcap#每日回报率＝每日总回报/总市值
#         print("nan_map[stk]=[date] isnan",nan_map)   
    return hml

In [ ]:
# btm_3group.keys()
# btm_3group["smallgroup"]

In [ ]:
#test for get_group_return
valid_total_market_cap={}
nan_map={}
# get_group_return(index_date,btm_3group["smallgroup"],return_data,df_market_cap,nan_map,valid_total_market_cap,recordnan_withstk)


In [ ]:
# return_data2=return_data.fillna(0)
# df_market_cap2=df_market_cap.fillna(0)

# return_data recordnan_withstk
smb=get_smb(index_date,stocks,return_data,df_market_cap,recordnan_withstk)
smb
# 2017-10-24   -0.022904
# 2017-11-24   -0.018079
# 2017-12-26   -0.000116
# 2018-01-26   -0.079003
# 2018-02-28    0.051783
# 2018-03-30    0.015532
# dd={"aa":11}
# dd[pd.Timestamp('2018-8-27')]=33
# pd.Timestamp('2018-8-27') in dd
# dd

In [ ]:
#recordnan_withdate
# smb2=get_smb(index_date,stocks,return_data,df_market_cap,recordnan_withdate)
# smb2


In [ ]:
# get_div_point(index_date,df_market_cap,btm2)
# np.percentile(list(btm.iloc[0]),30)
data_mode=""
#hml采用过滤后的股票列表
hml=get_hml(index_date,stocks,return_data,df_market_cap,btm,recordnan_withstk)

#btm不做调整的计算
# btm.fillna(0)
# hml=get_hml(index_date,stocks,return_data,df_market_cap,btm)
data_mode=data_mode_global
hml
# 2017-10-24   -0.011637
# 2017-11-24   -0.009626
# 2017-12-26   -0.013835
# 2018-01-26    0.044800
# 2018-02-28   -0.033500
# 2018-03-30   -0.034931


In [ ]:
#market (HuShen300)
def get_hs_market(data_starttime,data_endtime):
    s_hs_market=None
    if(data_mode=="net"):
        s_hs_market = get_price(['000300.XSHG'],start_date=data_starttime, end_date=data_endtime,frequency='1d',fields='close')
        # df_Rm_data=pd.read_csv("df_Rm_data.csv",index_col=0,header=None)
        type(df_hs_market)#pandas.core.series.Series
        s_hs_market
    else:
        s_hs_market=pd.read_csv("hs_market.csv",index_col=0,header=None)#
        #type(s_hs_market)==pandas.core.frame.DataFrame
    s_hs_market=trans_series(s_hs_market)    
    s_hs_market=trans_index(s_hs_market)
    return s_hs_market

In [ ]:
s_hs_market=get_hs_market(data_starttime,data_endtime)
s_hs_market#沪深股票的行情是升序的，所以之前算错了


In [ ]:
#保存smb hml 因子
# smb.to_csv("smb.csv")
# hml.to_csv("hml.csv")
# s_hs_market.to_csv("hs_market.csv")

In [ ]:
#处理无风险利率
def get_save_ratio():
    df_rmb_save_ratio=pd.read_csv("rmb_ratio_pure.csv",index_col=0,header=None)
    df_rmb_save_ratio=trans_index(df_rmb_save_ratio)
    # type(df_save_ratio.index[0])#pandas.tslib.Timestamp
    df_rmb_save_ratio=trans_series(df_rmb_save_ratio)
    return df_rmb_save_ratio
s_save_ratio=get_save_ratio()


In [ ]:
#market return(HuShen300) and risk free return(0.375724091% 1M, for I can't get this rate from Ricequant nor can I upload my own data)
def get_rm(index_date,s_hs_market):
    Rm = pd.Series(index = index_date)
    if(data_mode=="fast"):
        Rm=pd.read_csv("rm.csv",index_col=0,header=None) 
        Rm=trans_series(Rm)
    else:
        s_hs_market2=s_hs_market.reindex(index_date)
        Rm=s_hs_market2.pct_change()
    return Rm

def get_rf(index_date,s_save_ratio):
    Rf = pd.Series(index = index_date)
    Rf0=s_save_ratio.reindex(index=index_date,method="pad")
    Rf=Rf0/100/12
    return Rf

def get_index_r(index_date,s_index):
    s_r = pd.Series(index = index_date)
    s_index2=s_index.reindex(index=index_date)
    s_r=s_index2.pct_change()
    return s_r

In [ ]:
#change_mode
data_mode=""
Rm=get_rm(index_date,s_hs_market)
Rf=get_rf(index_date,s_save_ratio)
data_mode=data_mode_global

Rm_Rf=Rm-Rf
Rm_Rf
# Rf.to_csv("rf2.csv")
# s_save_ratio


In [ ]:
#save rm and rf data
# Rm.to_csv("rm.csv")
# Rf.to_csv("rf.csv")

In [ ]:
#引入大数据因子
s_bigdata=pd.read_csv("bigdata_pure.csv",index_col=0,header=None)
s_bigdata=trans_series(s_bigdata)
s_bigdata=trans_index(s_bigdata)

s_bigdata_r=get_index_r(index_date,s_bigdata)
s_bigdata_r
# s_bigdata=pd.to_numeric(s_bigdata)#处理不了含有千分位符号的数字字符串
# s=s_bigdata.apply(lambda x:float(x.replace(",","",2)))
# s.to_csv("bigdata_pure.csv")
# float("2,,333.00".replace(",","",3))
# s_bigdata_r.to_csv("bigdata_r.csv")


In [ ]:
#x = pd.DataFrame(index=index_date,columns=['Rm-Rf','SMB','HML'])
def get_df_X(index_date,factors):
    df_X=pd.DataFrame(factors,index=index_date)
    df_X = df_X[1:]
    return df_X
# df_X=get_df_X(index_date,{"Rm_Rf":Rm_Rf,'SMB':smb,'HML':hml,"bigdata":s_bigdata_r})
# df_X=get_df_X(index_date,{"Rm_Rf":Rm_Rf})

In [ ]:
df_X=get_df_X(index_date,{"Rm_Rf":Rm_Rf,'SMB':smb,'HML':hml,"bigdata":s_bigdata_r})
# df_X=get_df_X(index_date,{"Rm_Rf":Rm_Rf,'SMB':smb,'HML':hml})
# df_X.describe()

df_X.corr(method='pearson')
# df_X3=df_X.drop("Rm_Rf",axis=1)
# df_X3=df_X3.drop("bigdata",axis=1)
# df_X3.corr(method='pearson')

# df_X.plot()
# X=df_X.values
# X
# df_X.cov()

# 	HML	Rm_Rf	SMB	bigdata
# HML	1.000000	0.620621	-0.577628	0.047892
# Rm_Rf	0.620621	1.000000	-0.976422	0.560603
# SMB	-0.577628	-0.976422	1.000000	-0.590060
# bigdata	0.047892	0.560603	-0.590060	1.000000

In [ ]:
np.linspace(1,10,10)

In [ ]:
#get_stocks_r 计算某个数组的股票的收益率Series
#index_date 为观察的数据周期中的日期
#stocks 为要计算的股票数组
#return_data 为总的回报率数据
#df_market_cap 为总的市值数据
def get_stocks_r(index_date,stocks,return_data,df_market_cap):
    s_r = pd.Series(index = index_date)
#     median_size,lmark,hmark=get_div_point(index_date,df_market_cap,btm)
    for date in index_date:
        total_return=0.0
        total_cap=0.0
        for stk in stocks:
#             if(stk=="000022.XSHE"):#第一行为nan的原因，导致整个第一次的total_cap=0
#             print('----------',df_market_cap[stk][date],return_data[stk][date],stk,date)
            if(np.isnan(df_market_cap[stk][date]) or np.isnan(return_data[stk][date])):
                continue
            stock_date_cap=df_market_cap[stk][date]
            stock_date_return=return_data[stk][date]*stock_date_cap
            total_cap=total_cap+stock_date_cap
            total_return=total_return+stock_date_return

        if(total_cap!=0):
            s_r[date] = (total_return)/total_cap#每日回报率＝每日总回报/总市值
        else:
            s_r[date]=0
    return s_r

def get_yT(s_y): 
    y = np.array(s_y)
    y=y[1:]
    YT = y.T
    return YT
# y=get_y(return_data_ydyl,Rf)

In [ ]:
# return_data["000022.XSHE"]
Rf

In [ ]:
# str(concept('一带一路'))
arr_ydyl=['000022.XSHE', '000042.XSHE', '000065.XSHE', '000088.XSHE', '000090.XSHE', '000151.XSHE', '000157.XSHE', '000159.XSHE', '000400.XSHE', '000425.XSHE', '000498.XSHE', '000507.XSHE', '000528.XSHE', '000564.XSHE', '000582.XSHE', '000610.XSHE', '000617.XSHE', '000672.XSHE', '000680.XSHE', '000703.XSHE', '000777.XSHE', '000797.XSHE', '000852.XSHE', '000862.XSHE', '000877.XSHE', '000905.XSHE', '000928.XSHE', '000939.XSHE', '000978.XSHE', '600026.XSHG', '600028.XSHG', '600031.XSHG', '600068.XSHG', '600089.XSHG', '600105.XSHG', '600118.XSHG', '600125.XSHG', '600150.XSHG', '600170.XSHG', '600176.XSHG', '600179.XSHG', '600190.XSHG', '600202.XSHG', '600256.XSHG', '600279.XSHG', '600312.XSHG', '600317.XSHG', '600320.XSHG', '600339.XSHG', '600350.XSHG', '600368.XSHG', '600406.XSHG', '600425.XSHG', '600428.XSHG', '600449.XSHG', '600477.XSHG', '600487.XSHG', '600495.XSHG', '600509.XSHG', '600522.XSHG', '600528.XSHG', '600540.XSHG', '600580.XSHG', '600581.XSHG', '600583.XSHG', '600706.XSHG', '600717.XSHG', '600720.XSHG', '600778.XSHG', '600798.XSHG', '600801.XSHG', '600820.XSHG', '600888.XSHG', '600984.XSHG', '002040.XSHE', '600970.XSHG', '002047.XSHE', '002051.XSHE', '600017.XSHG', '600018.XSHG', '601872.XSHG', '601008.XSHG', '002135.XSHE', '601919.XSHG', '002146.XSHE', '601808.XSHG', '601857.XSHG', '601390.XSHG', '601866.XSHG', '002205.XSHE', '002207.XSHE', '601186.XSHG', '601668.XSHG', '002266.XSHE', '601766.XSHG', '601727.XSHG', '601989.XSHG', '002302.XSHE', '002307.XSHE', '002309.XSHE', '601618.XSHG', '601117.XSHG', '300011.XSHE', '002323.XSHE', '002353.XSHE', '601179.XSHG', '601106.XSHG', '002459.XSHE', '300090.XSHE', '300103.XSHE', '601018.XSHG', '002475.XSHE', '601028.XSHG', '002554.XSHE', '002738.XSHE', '601880.XSHG', '002524.XSHE', '300183.XSHE', '601789.XSHG', '300208.XSHE', '300262.XSHE', '601669.XSHG', '601800.XSHG', '603333.XSHG', '603308.XSHG', '300351.XSHE', '603111.XSHG', '600023.XSHG', '300523.XSHE', '300374.XSHE', '603969.XSHG', '603569.XSHG', '603338.XSHG', '603018.XSHG', '603169.XSHG', '002828.XSHE', '603036.XSHG', '603298.XSHG', '603966.XSHG', '603218.XSHG', '601228.XSHG', '600326.XSHG', '603588.XSHG', '603619.XSHG', '600219.XSHG', '300065.XSHE', '002742.XSHE', '600415.XSHG', '603871.XSHG', '300732.XSHE', '600338.XSHG', '600010.XSHG', '600039.XSHG', '601899.XSHG', '002800.XSHE', '601500.XSHG', '300589.XSHE']
if(data_type=="net"):
    arr_ydyl=concept('一带一路')
# print('len(arr_ydyl)',len(arr_ydyl))#157

return_data_ydyl=get_stocks_r(index_date,arr_ydyl,return_data,df_market_cap)
# return_data_ydyl.to_csv("return_data_ydyl.csv")
return_data_ydyl

y=return_data_ydyl-Rf
# yT=get_yT(y)
y
# 2018-03-30   -0.029865
# 2018-02-28   -0.105838

In [ ]:
def get_df_xy(index_date,factors):
    df_xy=pd.DataFrame(factors,index=index_date)
    df_xy = df_xy[:data_count-1]
    return df_xy


In [ ]:
df_xy=get_df_xy(index_date,{"y":y,"Rm_Rf":Rm_Rf,'SMB':smb,'HML':hml,"bigdata":s_bigdata_r})
# df_X=get_df_X(index_date,{"Rm_Rf":Rm_Rf,'SMB':smb,'HML':hml})
df_xy.index.name="date"
df_xy.describe()
# df_xy.corr(method='pearson')
# df_xy.plot()
# df_xy.values

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
# np.seterr(invalid='warn')
# ols_model = smf.ols('y ~ HML + Rm_Rf + np.log(SMB) ', data=df_xy)#股票序列数据不能用对数变换的，不可能收益都为正

# 4个因子
# ols_model = smf.ols('y ~ HML + Rm_Rf + SMB + bigdata', data=df_xy)

# 3个因子
# ols_model = smf.ols('y ~ HML + Rm_Rf + SMB ', data=df_xy)
# ols_model = smf.ols('y ~ HML + SMB ', data=df_xy)

#去掉市值
ols_model=smf.ols('y ~ HML + SMB + bigdata', data=df_xy)
results = ols_model.fit()#cov_type='HC3'

#去掉大数据
# results = smf.ols('y ~ Rm_Rf  + HML+ SMB  ', data=df_xy).fit()
# results = smf.ols('y ~ Rm_Rf  + HML ', data=df_xy).fit()
# results = smf.ols('y ~  Rm_Rf  ', data=df_xy).fit()
results.summary()

In [ ]:
# regression.linear_model.RegressionResults
# results.resid_pearson
# results.model.data.endog
# results.model.data.exog
# results.resid.plot()
plt.scatter(results.resid.index,results.resid)

In [ ]:
# import statsmodels.compat.lzip as lzip #No module named 'statsmodels.compat.lzip'
# 应该是lzip是compat的属性，而不是子模块，import 只能引入到模块级别，不能引入到属性活着功能级别
from statsmodels.compat import lzip
import statsmodels.iolib.table as table

def white_test(results):
    name = ['lm statistic', 'lm_pvalue','f-statistic',"f_pvalue"]
    # f-statistic of the hypothesis that the error variance does not depend on x. This is an alternative test variant not the original LM test.
    test = sms.het_white(results.resid, results.model.exog)
    ziped=lzip(name, test)
    # white=sms.het_white(results.resid,results.model.exog)
    white=table.SimpleTable(ziped)
    white.title="White Test"
#     print(white)
    return white
white_test(results)
#     print(white)
#             White Test            
# ==================================
# lm statistic   41.55454399712868  
#  lm_pvalue   3.963585550453876e-06
# f-statistic    7.343179599291278  
#   f_pvalue   7.650453944017435e-08
# ----------------------------------


# sms.het_white?

In [ ]:
#np.log(smb)#负数没有log，所以出现了问题，收益率不好用log来调整，都加1求log好像也不太好的方式，
# /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
#   """Entry point for launching an IPython kernel.
# /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
#   """Entry point for launching an IPython kernel.
# sms.linear_rainbow(results)#(0.835850366394606, 0.7279272220899624)

name = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
test = sms.het_breuschpagan(results.resid, results.model.exog)
ziped=lzip(name, test)
bp=table.SimpleTable(ziped)
bp
# sms.het_breuschpagan?

In [ ]:
# Goldfeld-Quandt test
name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(results.model.endog, results.model.exog,idx=2)
ziped=lzip(name, test)
gq=table.SimpleTable(ziped)
gq
# sms.het_goldfeldquandt?

In [ ]:
# YT=get_y(return_data_ydyl,Rf)
# # YT=get_y(return_data["600000.XSHG"],Rf)
# X = sm.add_constant(X)
# mod = regression.linear_model.OLS(YT, X).fit()
# p = mod.params
# print(p)
# mod.summary()
# #HML	Rm_Rf	SMB	bigdata

In [ ]:
#np.corrcoef(X)
# np.seterr(divide='ignore', invalid='ignore')
# np.corrcoef(X,rowvar=False)#终于调用成功了，原因时前面的X = sm.add_constant(X)改变了X，已经不是一个简单的3维数组啦，是4列了，而去第一列的数据是：1.00000000e+00所以报错了。
ols_model2=smf.ols('y ~ HML + SMB', data=df_xy)
results2 = ols_model2.fit()
results2.summary()

In [ ]:
name = ['lm statistic', 'lm_pvalue','f-statistic',"f_pvalue"]
# f-statistic of the hypothesis that the error variance does not depend on x. This is an alternative test variant not the original LM test.
test = sms.het_white(results2.resid, results2.model.exog)
ziped=lzip(name, test)
# white=sms.het_white(results.resid,results.model.exog)
white=table.SimpleTable(ziped)
white.title="White Test"
# results.model.exog
white

In [ ]:
# price_data=get_price_cs(data_starttime,data_endtime)#1944 rows × 3631 columns
# price_data

In [ ]:
import statsmodels.api as sm
# sm.formula==smf#True
# regression.linear_model.OLS??
# results.compare_lm_test(restricted={"model":{"ssr":results.ssr,"df_resid":results.df_resid}})

In [ ]:
# ols_model.whiten(results.resid)
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind,levene
from statsmodels.stats import anova
from statsmodels.stats.anova import anova_lm
# pearsonr??
# results
# levene(df_X.iloc[:,0],df_X.iloc[:,1])
anova_lm(results,test="Chisq")

In [ ]:
# anova_lm?

In [ ]:
 plt.hist(results.model.exog)

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df_xy)
# autocorrelation_plot(results.resid)

In [ ]:
# fig = plt.figure(figsize=(15,8))
# fig = sm.graphics.plot_regress_exog(results, df_X.columns, fig=fig)
# df_X[:,np.array([True,False,True])]
# np.array(df_X)==np.asarray(df_X)

# 大数据因子的各种处理
# df_bigdata_origin=pd.read_csv("bigdata.csv",encoding="gbk",index_col=0)
# df_bigdata_origin
# df_bigdata=pd.Series(df_bigdata_origin["收盘点位"],index=df_bigdata_origin.index,name="bigdata")
# # df_bigdata["close"]=df_bigdata_origin["收盘点位"]
# df_bigdata.index.name="index_date"
# df_bigdata.to_csv("bigdata_close.csv")
# df_bigdata=trans_index(df_bigdata)
# df_bigdata2=df_bigdata[df_bigdata.index.isin(index_date)]
# df_bigdata2.to_csv("bigdata_pure.csv")
# df_bigdata=pd.read_csv("bigdata_pure.csv",index_col=0,header=None)
# df_bigdata["test"]=df_bigdata_origin["收盘点位"]
# df_bigdata

In [ ]:
# dfx=df_X.copy()
# dfx["y"]=return_data_ydyl-Rf
# dfx["seq"]=np.arange(len(dfx))
# dfx.index.name="date"
# # dfx.reindex(index=["seq","date"],columns=["HML","Rm_Rf","SMB","bigdata"],copy=True)
# # dfx.index.append(dfx["seq"])
# df=pd.DataFrame(dfx[["HML","Rm_Rf","SMB","bigdata"]].values,index=[dfx["seq"],dfx.index],columns=["HML","Rm_Rf","SMB","bigdata"])
# df

In [ ]:
# price_data.apply(lambda x:x)
# np.percentile([1,2,np.nan,4,5],40)
# price_data.dropna(axis=1,how='all') 
# fm  = pd.fama_macbeth(y=df['y'],x=df[["HML","Rm_Rf","SMB","bigdata"]])
# fm
df_xy

In [ ]:
# df = pd.DataFrame({
#         'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
#         'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
#         'C': np.random.randn(8),
#         'D': np.random.randn(8)
#     })

# def fmreg(data,formula):
#     return smf.ols(formula,data=data).fit().params

# res=df_xy.groupby('date').apply(fmreg,'y~HML+Rm_Rf+SMB')

In [ ]:
# df.groupby("A").mean()
# ss=pd.Series([3,3,3,np.nan,44])
# type(ss.value_counts(dropna=1))
# type(pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C']))#pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

In [ ]:
# ?smf.wls()
ols_model3=smf.wls('y ~ HML + SMB +bigdata', data=df_xy)
results3 = ols_model3.fit()#cov_type='HC3'
results3.summary()

In [ ]:
white_test(results3)

In [ ]:
import time
time.time()
time.strftime()